In [16]:
project_name = "Test_numbereval"
run_name = "Eval_run"


In [17]:
load_path = "/DATA3/vaibhav/isign/PretrainingISL/helpers/predictions_new/Test_numbereval_numbers_best_model_checkpoint.pth"

In [18]:
import os
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

eval_df = pd.read_csv('/DATA3/vaibhav/isign/PretrainingISL/helpers/test_numbers.csv')


In [19]:
import re
import numpy as np
import torch
import wandb
import random
import gc
import collections
import math
import ast
import collections
import math
import sacrebleu

from tqdm import tqdm
from transformers import (
    BertConfig, BertModel,
    GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
    EncoderDecoderModel,
    PreTrainedTokenizerFast,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    get_constant_schedule_with_warmup
)
from datasets import Dataset
from torch.utils.data import DataLoader
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from bleu_cal import quick_bleu_metric
from dataloaders import FeatureVectorDataset, FeatureVectorDataset_Isign
from pose_format import Pose
from pose_format.pose_visualizer import PoseVisualizer
from itertools import cycle

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set random seed for reproducibility
set_seed()

def get_threshold(current_step, total_steps):
    if total_steps == 0:
        return 1.1
    else:
        return min(current_step / total_steps, 0.85) # Changed from 0.9

#Hyperparameters here now 
learning_rate = 3e-4 #3e-4 
num_encoder_layers = 4 #4
num_decoder_layers = 4 #4
encoder_hidden_size = 512 #512
decoder_hidden_size = 512 #512
num_attention_heads = 8
dropout = 0.1
MAX_FRAMES = 300  # Max video frames.
max_position_embeddings_encoder = MAX_FRAMES
num_beams = 3
#label_smoothing = 0.1 not used yet
warmup_steps_ratio = 0.1
batch_size = 16 #64 #256
#gradient_accumulation_steps = 1 not used yet
lr_scheduler_type = 'warmup_linear_constant_afterwards'
num_epochs = 1
max_length_decoder = 128
vocab_size_decoder = 15000
num_keypoints = 152 # We have cherrypicked these
WEIGTH_DECAY = 0.01
randomize_word_order = False
steps_for_100percentIsign = 120000

POSE_DIR = "/DATA7/vaibhav/tokenization/CISLR/CISLR_v1.5-a_videos_poses/"
POSE_DIR_ISIGN = "/DATA3/vaibhav/isign/PretrainingISL/helpers/videos"
STEP_FRAMES = None  # Random sampling of frames.
STEP_FRAMES_ISIGN = None
STEP_FRAMES_CISLR = 4
ADD_NOISE_ISIGN = False
ADD_NOISE_CISLR = True


hyperparameters = {'learning_rate': learning_rate, 
                     'num_encoder_layers': num_encoder_layers,
                        'num_decoder_layers': num_decoder_layers,
                        'encoder_hidden_size': encoder_hidden_size,
                        'decoder_hidden_size': decoder_hidden_size,
                        'num_attention_heads': num_attention_heads,
                        'dropout': dropout,
                        'max_position_embeddings_encoder': max_position_embeddings_encoder,
                        'num_beams': num_beams,
                        'warmup_steps_ratio': warmup_steps_ratio,
                        'batch_size': batch_size,
                        'lr_scheduler_type': lr_scheduler_type,
                        'num_epochs': num_epochs,
                        'max_length_decoder': max_length_decoder,
                        'Max_frames_videos': MAX_FRAMES,
                        'vocab_size_decoder': vocab_size_decoder,
                        'num_keypoints': num_keypoints,
                        'POSE_DIR': POSE_DIR,
                        'POSE_DIR_ISIGN': POSE_DIR_ISIGN,
                        'randomize_word_order': randomize_word_order,
                        'steps_for_100percentIsign': steps_for_100percentIsign,
                        'ADD_NOISE_ISIGN': ADD_NOISE_ISIGN,
                        'ADD_NOISE_CISLR': ADD_NOISE_CISLR,
                        'Step_frames_sampling': STEP_FRAMES,
                        'Step_frame_isign': STEP_FRAMES_ISIGN,
                        'Step_frame_cislr': STEP_FRAMES_CISLR}



In [20]:
wandb.init(project=project_name, name=run_name, config = hyperparameters)

In [21]:
#Load the tokenizer as a PreTrainedTokenizerFast
tokenizer_target = PreTrainedTokenizerFast(tokenizer_file="tokenizer_file/target_tokenizer_numbers.json")
tokenizer_target.add_special_tokens({
    "bos_token": "<s>",
    "eos_token": "</s>",
    "unk_token": "<unk>",
    "pad_token": "<pad>",
    "mask_token": "<mask>",
    'additional_special_tokens': ['<PERSON>', '<UNKNOWN>']
})



2

In [22]:
print('Extracting video UIDs and labels...')
eval_video_uids = eval_df['uid_list'].apply(ast.literal_eval).tolist()



print('Appending <s> and </s> to labels...')
eval_labels = [f'<s>{text}</s>' for text in eval_df['text'].tolist()]



#train_labels = tokenizer_target(train_labels, max_length=max_length_decoder, padding="max_length", truncation=True)['input_ids']
eval_labels = tokenizer_target(eval_labels, max_length=max_length_decoder, padding="max_length", truncation=True)['input_ids']

eval_dataset = FeatureVectorDataset_Isign(eval_video_uids, tokenizer_target, 
                                        MAX_FRAMES, POSE_DIR_ISIGN, eval_labels, 
                                        step_frames=STEP_FRAMES_ISIGN, add_noise = ADD_NOISE_ISIGN)



# Create DataLoaders
print('Creating DataLoaders...')

eval_loader = DataLoader(eval_dataset, batch_size=batch_size, num_workers=2, pin_memory=True, prefetch_factor=2)




Extracting video UIDs and labels...
Appending <s> and </s> to labels...
Creating DataLoaders...


In [23]:
sub_project_name = "eval_numbers"
# Step 4: Define the Encoder and Decoder Models
# Encoder Configuration and Model
encoder_config = BertConfig(
    hidden_size=encoder_hidden_size,
    num_hidden_layers=num_encoder_layers,
    num_attention_heads=num_attention_heads,
    hidden_dropout_prob=dropout,  # Dropout after fully connected layers
    attention_probs_dropout_prob=dropout,  # Dropout on attention weights
)
#encoder = BertForCausalLM(encoder_config)
encoder = BertModel(encoder_config)
print(encoder_config)

# Decoder Configuration and Model
decoder_config = GPT2Config(
    vocab_size=len(tokenizer_target),
    n_positions=max_length_decoder, # We have padded and truncated to 128
    n_embd=decoder_hidden_size,
    n_layer=num_decoder_layers,
    n_head=num_attention_heads,
    pad_token_id=tokenizer_target.pad_token_id,
    bos_token_id=tokenizer_target.bos_token_id,
    eos_token_id=tokenizer_target.eos_token_id,
    add_cross_attention=True,  # Important for Seq2Seq models (Can't find this on HF docs)
    embd_pdrop=dropout,  # Dropout on embeddings 
    attn_pdrop=dropout,  # Dropout on attention probabilities 
    resid_pdrop=dropout  # Dropout on residual connections 
)
print(decoder_config)
decoder = GPT2LMHeadModel(decoder_config)

########################################################
#decoder.resize_token_embeddings(len(tokenizer_target))
########################################################

# Linear layer to project feature vectors to the expected input shape
class FeatureProjection(torch.nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims = 1024):
        super(FeatureProjection, self).__init__()
        # self.linear = torch.nn.Linear(input_dim, output_dim)
        self.linear1 = torch.nn.Linear(input_dim, hidden_dims)
        self.linear2 = torch.nn.Linear(hidden_dims, output_dim)
        self.gelu = torch.nn.GELU()

    def forward(self, x):
        # return self.linear(x)
        x = self.gelu(self.linear1(x))
        x = self.linear2(x)
        return x


# Combine Encoder and Decoder into EncoderDecoderModel
feature_projection = FeatureProjection(num_keypoints, encoder_config.hidden_size)
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

########################################################################
#model.decoder.resize_token_embeddings(len(tokenizer_target))
########################################################################

# Tie weights (optional)
model.config.decoder_start_token_id = tokenizer_target.bos_token_id
model.config.eos_token_id = tokenizer_target.eos_token_id
model.config.pad_token_id = tokenizer_target.pad_token_id
model.config.vocab_size = decoder_config.vocab_size
model.config.max_length = max_length_decoder

# After creating your model and adding special tokens


# Load the best model checkpoint if it exists
# Create directory if it does not exist
if not os.path.exists('load_pretrained'):
    os.makedirs('load_pretrained')

if not os.path.exists('predictions_new'):
    os.makedirs('predictions_new')

# load_path = ""
checkpoint_path = "predictions_new/"+project_name+'_'+sub_project_name+'_'+"checkpoint.pth"
best_checkpoint_path = "predictions_new/"+project_name+'_'+sub_project_name+'_'+"best_model_checkpoint.pth"
best_checkpoint_path_isignB1 = "predictions_new/"+project_name+'_'+sub_project_name+'_'+"best_model_checkpoint_isign.pth"
best_checkpoint_path_isignB4 = "predictions_new/"+project_name+'_'+sub_project_name+'_'+"best_model_checkpoint_isignB4.pth"
def save_checkpoint(model, feature_projection, optimizer, scheduler, epoch, best_val_B4, best_val_loss, checkpoint_path, current_step,
                    best_val_B4_isign, best_val_loss_isign, best_val_B1_isign, epoch_steps):
    checkpoint = {
        'epoch': epoch,
        'current_step' : current_step,
        'model_state_dict': model.state_dict(),
        'feature_projection_state_dict': feature_projection.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_B4': best_val_B4,
        'best_val_loss': best_val_loss,
        'best_val_B4_isign': best_val_B4_isign,
        'best_val_loss_isign': best_val_loss_isign,
        'best_val_B1_isign': best_val_B1_isign,
        'epoch_steps': epoch_steps
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at epoch {current_step}")
    print("*"*50)

def load_checkpoint(model, feature_projection, optimizer, scheduler, checkpoint_path):
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        feature_projection.load_state_dict(checkpoint['feature_projection_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        current_step = checkpoint['current_step']
        best_val_B4 = checkpoint['best_val_B4']
        best_val_loss = checkpoint.get('best_val_loss', float('inf'))  # Backwards compatibility
        best_val_B4_isign = checkpoint['best_val_B4_isign']
        best_val_B1_isign = checkpoint['best_val_B1_isign']
        best_val_loss_isign = checkpoint.get('best_val_loss_isign', float('inf'))  # Backwards compatibility
        epoch_steps = checkpoint['epoch_steps']
        print(f"Checkpoint loaded, resuming from epoch {start_epoch}")
        print("*"*50)
        return start_epoch, best_val_B4, best_val_loss, best_val_B4_isign, best_val_loss_isign, best_val_B1_isign, epoch_steps
    else:
        print("No checkpoint found, starting from scratch")
        return 0, 0.0, float('inf')


# Initialize model, optimizer, and scheduler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
feature_projection.to(device)

optimizer = torch.optim.AdamW(
    list(model.parameters()) + list(feature_projection.parameters()),
    weight_decay=WEIGTH_DECAY,
    lr=learning_rate
)

# Calculate total steps for scheduler

# total_steps = len(train_loader) * num_epochs
warmup_steps = 100

# Create scheduler with linear warmup and constant afterwards

scheduler = get_constant_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    #num_training_steps=total_steps  # Will maintain constant lr after warmup
)

#wandb.watch(model, log="all", log_freq=100)

epoch_steps = 0
# Load checkpoint or pretrained weights

#/DATA3/vaibhav/isign/PretrainingISL/predictions_new/CISLR_Pretraining_FrameMatch_Linear60kBPE0.85Threshold_PT1_best_model_checkpoint_isignB4.pth                
if os.path.exists(""): #best_checkpoint_path_isignB4
    start_epoch, best_val_B4, best_val_loss, best_val_B4_isign, best_val_loss_isign, best_val_B1_isign, epoch_steps = load_checkpoint(
        model, feature_projection, optimizer, scheduler, best_checkpoint_path_isignB4
    )
    start_epoch = 0
    print("Loaded best model checkpoint IsignB4")
    print("*"*50)

elif os.path.exists(load_path):
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    feature_projection.load_state_dict(checkpoint['feature_projection_state_dict'])
    
    # Load optimizer and scheduler states but reset epoch counter
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    # Reset epoch and metrics but keep scheduler state
    start_epoch = 0
    best_val_B4 = 0.0
    best_val_B4_isign = 0.0
    best_val_B1_isign = 0.0
    best_val_loss_isign = float('inf')
    best_val_loss = float('inf')
    print("Loaded pretrained model")
    print("*"*50)
else:
    print("No checkpoint or pretrained weights found, starting from scratch")
    print("*"*50)
    start_epoch = 0
    best_val_B4 = 0.0
    best_val_B4_isign = 0.0
    best_val_B1_isign = 0.0
    best_val_loss_isign = float('inf')
    best_val_loss = float('inf')


BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 512,
  "n_head": 8,
  "n_inner": null,
  "n_layer": 4,
  "n_positions": 128,
  "pad_token_id": 1,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inv

/tmp/ipykernel_417967/719206466.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path)


Loaded pretrained model
**************************************************


In [24]:
epoch = 1
def model_eval(eval_loader, log_what, best_val_B4,best_val_loss,best_val_B4_isign,
               best_val_B1_isign,best_val_loss_isign,counter, current_step,epoch_steps, save_model=False):
    model.eval()
    feature_projection.eval()
    eval_loss = 0.0
    all_refs = []
    sacre_refs = []
    sacre_preds = []
    all_preds = []
    
    with torch.no_grad():
        eval_progress = tqdm(eval_loader, desc=f"Evaluating Epoch {epoch+1}")
        for eval_batch in eval_progress:
            input_ids = eval_batch['input_ids'].to(device)
            attention_mask = eval_batch['attention_mask'].to(device)
            labels = eval_batch['labels'].to(device)
            
            input_ids = feature_projection(input_ids)
            input_ids = input_ids.view(input_ids.size(0), -1, encoder_config.hidden_size)

            outputs = model(
                inputs_embeds=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            eval_loss += outputs.loss.item()
            
            # Generate predictions with improved parameters
            generated_ids = model.generate(
                inputs_embeds=input_ids,
                attention_mask=attention_mask,
                max_length=max_length_decoder,
                num_beams=num_beams,
                length_penalty=0.6,
                no_repeat_ngram_size=3,
                early_stopping=True
            )
            
            # Process predictions and references
            generated_ids = torch.where(
                generated_ids == -100,
                torch.tensor(tokenizer_target.pad_token_id).to(generated_ids.device),
                generated_ids
            )
            labels = torch.where(
                labels == -100,
                torch.tensor(tokenizer_target.pad_token_id).to(labels.device),
                labels
            )
            
            preds = tokenizer_target.batch_decode(generated_ids, skip_special_tokens=True)
            refs = tokenizer_target.batch_decode(labels, skip_special_tokens=True)
            
            for ref in refs:
                sacre_refs.append(str(ref))
            
            for pred in preds:
                sacre_preds.append(str(pred))
            
            ref_tokens = [ref.strip().split() for ref in refs]
            pred_tokens = [pred.strip().split() for pred in preds]
            
            all_refs.extend([ref] for ref in ref_tokens)
            all_preds.extend(pred_tokens)
    
    # Calculate metrics
    avg_eval_loss = eval_loss / len(eval_loader)
    bleu1, bleu2, bleu3, bleu4 = quick_bleu_metric(all_refs, all_preds, split=f'{log_what }Validation')
    bleu_sacre = sacrebleu.corpus_bleu(sacre_preds, [sacre_refs])
    bleu_sacre1, bleu_sacre2, bleu_sacre3, bleu_sacre4 =  bleu_sacre.precisions[0], bleu_sacre.precisions[1], bleu_sacre.precisions[2], bleu_sacre.precisions[3]
    # Save best model
    # Log metrics
    if log_what == "CISLR":

        print(f'Sacre Bleu1_CISLR :{bleu_sacre1}')
        print(f'Sacre Bleu2_CISLR :{bleu_sacre2}')
        print(f'Sacre Bleu3_CISLR :{bleu_sacre3}')
        print(f'Sacre Bleu4_CISLR :{bleu_sacre4}')
        if bleu4 > best_val_B4 or (bleu4 == best_val_B4 and avg_eval_loss < best_val_loss):
            best_val_B4 = bleu4
            best_val_loss = avg_eval_loss
            print('Saving CISLR best model checkpoint...')
            save_checkpoint(
                model, feature_projection, optimizer, scheduler,
                epoch, best_val_B4, best_val_loss, best_checkpoint_path, 
                current_step, best_val_B4_isign, best_val_loss_isign, best_val_B1_isign, epoch_steps
            )
            
            df = pd.DataFrame({
                'Reference': [' '.join(ref[0]) for ref in all_refs],
                'Prediction': [' '.join(pred) for pred in all_preds]
            })
            df.to_csv(f'predictions_new/{project_name}_{sub_project_name}_predictions{log_what}.csv', index=False)
        
        wandb.log({
            'epoch': epoch + 1,
            'val/eval_loss': avg_eval_loss,
            'val/bleu1': bleu1 * 100,
            'val/bleu2': bleu2 * 100,
            'val/bleu3': bleu3 * 100,
            'val/bleu4': bleu4 * 100
        })
        wandb.log({
            'val/bleu1_sacre': bleu_sacre1,
            'val/bleu2_sacre': bleu_sacre2,
            'val/bleu3_sacre': bleu_sacre3,
            'val/bleu4_sacre': bleu_sacre4
        })
    elif log_what == "ISIGN":
        print(f'Sacre Bleu1_Isign :{bleu_sacre1}')
        print(f'Sacre Bleu2_Isign :{bleu_sacre2}')
        print(f'Sacre Bleu3_Isign :{bleu_sacre3}')
        print(f'Sacre Bleu4_Isign :{bleu_sacre4}')
        if counter >= 1:
            if bleu4 > best_val_B4_isign or (bleu4 == best_val_B4_isign and avg_eval_loss < best_val_loss_isign):
                best_val_B4_isign = bleu4
                best_val_loss_isign = avg_eval_loss
                print('Saving IsignB4 best model checkpoint...')
                save_checkpoint(
                    model, feature_projection, optimizer, scheduler,
                    epoch, best_val_B4, best_val_loss, best_checkpoint_path_isignB4, 
                    current_step, best_val_B4_isign, best_val_loss_isign, best_val_B1_isign, epoch_steps
                )
                
                df = pd.DataFrame({
                    'Reference': [' '.join(ref[0]) for ref in all_refs],
                    'Prediction': [' '.join(pred) for pred in all_preds]
                })
                df.to_csv(f'predictions_new/{project_name}_{sub_project_name}_predictions{log_what}B4.csv', index=False)
            
            if bleu1 > best_val_B1_isign or (bleu1 == best_val_B1_isign and avg_eval_loss < best_val_loss_isign):
                best_val_B1_isign = bleu1
                best_val_loss_isign = avg_eval_loss
                print('Saving IsignB1 best model checkpoint...')
                save_checkpoint(
                    model, feature_projection, optimizer, scheduler,
                    epoch, best_val_B4, best_val_loss, best_checkpoint_path_isignB1, 
                    current_step, best_val_B4_isign, best_val_loss_isign,best_val_B1_isign, epoch_steps
                )
                
                df = pd.DataFrame({
                    'Reference': [' '.join(ref[0]) for ref in all_refs],
                    'Prediction': [' '.join(pred) for pred in all_preds]
                })
                df.to_csv(f'predictions_new/{project_name}_{sub_project_name}_predictions{log_what}B1.csv', index=False)
            
        wandb.log({
            'val/eval_loss_isign': avg_eval_loss,
            'val/bleu1_isign': bleu1 * 100,
            'val/bleu2_isign': bleu2 * 100,
            'val/bleu3_isign': bleu3 * 100,
            'val/bleu4_isign': bleu4 * 100
        })
        wandb.log({
            'val/bleu1_sacre_isign': bleu_sacre1,
            'val/bleu2_sacre_isign': bleu_sacre2,
            'val/bleu3_sacre_isign': bleu_sacre3,
            'val/bleu4_sacre_isign': bleu_sacre4
        })
    
    # Clean up memory
    torch.cuda.empty_cache()
    gc.collect()
    
    # Resume training
    model.train()
    feature_projection.train()
    return best_val_B4, best_val_loss, best_val_B4_isign, best_val_B1_isign, best_val_loss_isign



In [26]:
counter = 1
best_val_B4, best_val_loss, best_val_B4_isign, best_val_B1_isign, best_val_loss_isign = model_eval(
                eval_loader, "CISLR", best_val_B4, best_val_loss, best_val_B4_isign, 
                best_val_B1_isign, best_val_loss_isign, counter, epoch_steps, epoch_steps,save_model=True)
            

Evaluating Epoch 2:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]


BLEU CISLRValidation >>> B1:28.57, B2:17.82, B3:12.84, B4:11.52
Sacre Bleu1_CISLR :25.0
Sacre Bleu2_CISLR :5.882352941176471
Sacre Bleu3_CISLR :3.5714285714285716
Sacre Bleu4_CISLR :2.272727272727273


In [ ]:

# Step 7: Training and Evaluation Loop with BLEU Tracking
# Training and Evaluation Loop
for epoch in range(start_epoch, num_epochs):
    # Training phase
    model.train()
    feature_projection.train()
    train_loss = 0.0
    
    counter = 0
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")
    
    ## Hacky way to get infinite data.
    for batch in progress_bar:
        if epoch_steps % 1000 == 0:
            print(f"Training_step: {epoch_steps}")
            print(f'Bett MT val B4: {best_val_B4}')
        optimizer.zero_grad()
        
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        input_ids = feature_projection(input_ids)
        input_ids = input_ids.view(input_ids.size(0), -1, encoder_config.hidden_size)

        outputs = model(
            inputs_embeds=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(
            list(model.parameters()) + list(feature_projection.parameters()),
            max_norm=1.0
        )
        wandb.log({
            "learning_rate": scheduler.get_last_lr()[0],
            "step": epoch_steps
        })
        

        optimizer.step()
        scheduler.step()
        
        epoch_steps += 1
        train_loss += loss.item()
        #progress_bar.set_postfix({'Loss': loss.item()})
        
        # Evaluation phase (every 1000 steps)
        if epoch_steps % 2500 == 0:
            counter += 1
            best_val_B4, best_val_loss, best_val_B4_isign, best_val_B1_isign, best_val_loss_isign = model_eval(
                eval_loader, "CISLR", best_val_B4, best_val_loss, best_val_B4_isign, 
                best_val_B1_isign, best_val_loss_isign, counter, epoch_steps, epoch_steps,save_model=True)
            best_val_B4, best_val_loss, best_val_B4_isign, best_val_B1_isign, best_val_loss_isign = model_eval(
                eval2_loader, "ISIGN", best_val_B4,best_val_loss, best_val_B4_isign, 
                best_val_B1_isign, best_val_loss_isign,counter,  epoch_steps, epoch_steps, save_model=True)
        # if epoch_steps % 7500 == 0:
        #     # Save regular checkpoint
        #     save_checkpoint(
        #         model, feature_projection, optimizer, scheduler,
        #         epoch, best_val_B4, best_val_loss,
        #         "predictions_new/"+project_name+'_'+sub_project_name+'_'+str(epoch_steps)+"checkpoint.pth",
        #         epoch_steps
        #     )

    # End of epoch
    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs} - Average Training Loss: {avg_train_loss:.4f}')
    current_lr = optimizer.param_groups[0]['lr']
    wandb.log({'epoch': epoch+1, 'train/train_loss': avg_train_loss, 'learning_rate': current_lr})
    
    # Save regular checkpoint
    save_checkpoint(
        model, feature_projection, optimizer, scheduler,
        epoch, best_val_B4, best_val_loss, checkpoint_path,epoch_steps, best_val_B4_isign, best_val_loss_isign, best_val_B1_isign
    )


In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/DATA3/vaibhav/isign/PretrainingISL/helpers/isign_new_filtered.csv')

# Function to check if a string contains only numbers
def contains_only_numbers(text):
    # Split the text into words and check if all words are numeric
    # Also handles decimal points and spaces
    return all(word.replace('.','').isdigit() for word in str(text).strip().split())

# Filter rows where 'text' column contains only numbers
df_numbers = df[df['text'].apply(contains_only_numbers)]

# Save filtered DataFrame to new CSV if needed
df_numbers.to_csv('numbers_only_output.csv', index=False)

# Preview the results
print(f"Original rows: {len(df)}")
print(f"Rows with only numbers: {len(df_numbers)}")
print("\nFirst few rows of filtered data:")
print(df_numbers.head())

In [ ]:
import pandas as pd

def percentage_numbers(text):
    # Split the text into words
    words = str(text).strip().split()
    if not words:  # Handle empty strings
        return 0
    
    # Count words that are numbers
    num_count = sum(1 for word in words if word.replace('.','').isdigit())
    
    # Calculate percentage
    percentage = (num_count / len(words)) * 100
    return percentage

# Read the CSV file
df = pd.read_csv('/DATA3/vaibhav/isign/PretrainingISL/helpers/isign_new_filtered.csv')

# # Filter rows with more than 50% numbers
# threshold = 1
# df_numbers = df[df['text'].apply(percentage_numbers) > threshold]

# Preview results
# print(f"Original rows: {len(df)}")
# print(f"Rows with > {threshold}% numbers: {len(df_numbers)}")
# print("\nFirst few rows with percentages:")
# for idx, row in df_numbers.head().iterrows():
#     print(f"Text: {row['text']}")
#     print(f"Percentage numbers: {percentage_numbers(row['text']):.1f}%\n")

# # Save to CSV if needed
# df_numbers.to_csv('numbers_majority_output.csv', index=False)

Original rows: 0
Rows with > 1% numbers: 0

First few rows with percentages:


In [3]:
print(df_numbers.head())

Empty DataFrame
Columns: []
Index: []


In [3]:
import pandas as pd
test_df = pd.DataFrame({'text': ['1 0 4 5', '1 2 3 4', '2 0 2 0'], 'uid_list': [['1045'], ['1234'], ['2020']]})
test_df.to_csv('test_numbers.csv', index=False)